In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.8 MB/s eta 0:00:00


In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, dcc, html, Input, Output, callback
from datetime import date
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('/content/mydata2.csv')
rad_columns = ["stability",	"rights",	"health",	"safety",	"climate",	"cost",	"popularity"]
range = [0,10,20,30,40,50,60,70,80,90,100]

# initialize app
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)
server = app.server
#define layout and elements
app.layout = html.Div([
    html.H2("Oh the Places You'll Go!"),
    html.P("The following data come from a variety of reputable sources "
           "for each different metric: Some are from the World Bank, other metrics "
           "are compiled from the United Nations, and others are taken from the "
           "Institute for Economics & Peace. The dataset has then been uploaded "
           "to Kaggle and is now accessible to all Kaggle users."),
    html.H4("Select two countries"),
    dcc.Dropdown(
                    id='country-dropdown',
                   options=[{'label': country, 'value': country} for country in df['country']],
                    value=df['country'][:2],
                    multi=True
                ),
        html.Div([
        dcc.Graph(id='polar-plot', figure={})
    ], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([
        html.H4("Select two metrics"),
        dcc.Dropdown(
            id='metric-dropdown',
            options=[{'label': col, 'value': col} for col in df.columns if col != 'country'],
            value=["climate", "health"],
            multi=True
        ),
        dcc.Graph(id='bar-chart', figure={}),
    ], style={'display': 'inline-block', 'width': '50%', 'vertical-align': 'top', 'float': 'right'}),
    html.Div(id='country-info', style={'text-align': 'right', 'margin-right': '100px', 'margin-top':'100px'})
])

@app.callback(
   [Output('polar-plot', 'figure'),
    Output('bar-chart', 'figure')],
   [Input('country-dropdown', 'value'),
    Input('metric-dropdown','value')]
)

def update_plot(selected_countries, selected_metrics):
    traces = []
    for country in selected_countries:
        country_data = df[df['country'] == country]
        values = country_data[rad_columns].values.flatten().tolist()
        traces.append(go.Scatterpolar(
            r=values,
            theta=rad_columns,
            fill='toself',
            name=country
        ))
    layout = go.Layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        showlegend=True
    )
    polar_fig = go.Figure(data=traces, layout=layout)

    bar_traces = []
    for metric in selected_metrics:
        bar_traces.append(go.Bar(
            x=selected_countries,
            y=df[df['country'].isin(selected_countries)][metric],
            name=metric
        ))
    bar_layout = go.Layout(
        barmode='group',
        xaxis=dict(title='Country'),
        yaxis=dict(title='Value')
    )
    bar_fig = go.Figure(data=bar_traces, layout=bar_layout)

    return polar_fig, bar_fig

@app.callback(
    Output('country-info', 'children'),
    Input('country-dropdown', 'value')
)
def update_country_info(selected_countries):
    info = html.Div(children=[
        html.Div([
            html.H3(country),
            html.P(f"Strength: {df[df['country'] == country][rad_columns].max().idxmax()}"),
            html.P(f"Weakness: {df[df['country'] == country][rad_columns].min().idxmin()}")
        ], style={'display': 'inline-block', 'width': '25%', 'margin-bottom': '20px'})
        for country in selected_countries
    ])

    return info

# run app
if __name__ == '__main__':
    app.run(jupyter_mode='external', debug=True)

Dash app running on:


<IPython.core.display.Javascript object>